In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path = '/content/drive/My Drive/Colab Notebooks/Research Techniques II/merged_dataset_v2.csv'

In [3]:
import pandas as pd

df = pd.read_csv(file_path, index_col = 0)
df.head()

,Unnamed: 0,text,labels
0,0,Bu tətbiq niyə nömrəyə kod göndərmirr,0
1,1,Biz evin qida məhsullarini bravodan alırıq çox...,1
2,2,Pisdir çox exlaqsizliq bas alıb gedir,0
3,3,Qulag asmiram amma merifetli qardasdi Allah ca...,1
4,4,m10 kimi rahat gözəl tətbiq yoxdur,1


In [4]:
az_stopwords = set([
    'və', 'bu', 'bir', 'o', 'amma', 'həm', 'ki', 'belə', 'ilə', 'hər',
    'onun', 'mən', 'sən', 'biz', 'siz', 'onlar', 'onunla', 'də', 'da', 'gibi',
    'kimi', 'yəni', 'indi', 'heç', 'beləliklə', 'artıq', 'elə', 'çünki', 'axı',
    'nə', 'niyə', 'hara', 'necə', 'kim', 'nədən', 'nədir', 'ya', 'vəya', 'bütün',
    'lakin', 'sonra', 'əvvəl', 'ona', 'ondan', 'orada', 'burada', 'abs', 'aha',
    'oralarda', 'buralarda', 'bunlar', 'kimin', 'bizim', 'sizin', 'sənin',
    'mənim', 'kimə', 'nəyə', 'belədir', 'daha', 'hələ', 'dəfələrlə', 'ala',
    'biraz', 'idi', 'imiş', 'aa', 'aaa', 'abb', 'abdullayev', 'abi', 'amk'
    'arasında', 'haqqında', 'öz', 'özləri', 'özümüz', 'ah', 'ancaq', 'ancag'
    'yenə', 'arada', 'tezliklə', 'iki', 'üç', 'ancağ', 'aq', 'ayrıca',
    'dörd', 'beş', 'altı', 'yeddi', 'səkkiz', 'doqquz', 'on', 'qədər'
])

In [5]:
import re

# Emoji temizleyici
def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # yüz ifadeleri
        u"\U0001F300-\U0001F5FF"  # semboller & simgeler
        u"\U0001F680-\U0001F6FF"  # taşıma araçları
        u"\U0001F1E0-\U0001F1FF"  # bayraklar
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Temizleme ve tokenizasyon fonksiyonu
def clean_and_tokenize(text):
    text = remove_emojis(text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)            # Noktalama temizliği
    text = re.sub(r'\d+', '', text)                # Sayı temizliği
    text = re.sub(r'\s{2,}', ' ', text).strip()    # Fazla boşlukları temizle
    tokens = text.split()

    tokens = [t for t in tokens if len(t) > 1 and t not in az_stopwords]

    return tokens

# Uygulama
df = df.copy()
df['tokens'] = df['text'].apply(clean_and_tokenize)
df['text_cleaned'] = df['tokens'].apply(lambda x: ' '.join(x))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
X = vectorizer.fit_transform(df['text_cleaned'])

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# TF-IDF vektörizer (daha önce fit edildiğini varsayıyoruz)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=10000)
X = vectorizer.fit_transform(df['text_cleaned'])

# Vocabulary’yi sıraya koy
vocab_items = sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1])
vocab_df = pd.DataFrame(vocab_items, columns=['Kelime/N-gram', 'Sütun Indexi'])

# İlk 500 kelimeyi yazdır
for i, row in vocab_df.head(500).iterrows():
    print(f"{i+1}. {row['Kelime/N-gram']}")

1. abidə
2. abune
3. abunə
4. abş
5. abşda
6. abşın
7. ac
8. aca
9. aca bilmirəm
10. acaba
11. acilir
12. aciq
13. aclıq
14. acmaq
15. acmaq olmur
16. acmır
17. acı
18. acılmır
19. acından
20. ad
21. ada
22. adam
23. adam var
24. adama
25. adamam
26. adamda
27. adamdı
28. adamdır
29. adamin
30. adamla
31. adamlar
32. adamlar var
33. adamlara
34. adamları
35. adamların
36. adamı
37. adamı əsəb
38. adamın
39. addım
40. addımdır
41. adi
42. adicə
43. adim
44. adima
45. adin
46. adina
47. adinda
48. adini
49. adlar
50. adları
51. adli
52. adlı
53. admin
54. adminin
55. adrenalin
56. adresi
57. adı
58. adım
59. adıma
60. adımı
61. adın
62. adına
63. adında
64. adından
65. adını
66. adət
67. adətən
68. aferin
69. afrika
70. afrikada
71. afərin
72. afərin əkbərli
73. ag
74. ag ekran
75. agarib
76. agarir
77. agil
78. agilli
79. agzina
80. ai
81. aid
82. aid deyil
83. aid olan
84. aiddir
85. aile
86. ailə
87. ailədə
88. ailəsi
89. ailəyə
90. aktiv
91. aktivdir
92. aktyor
93. aktyordur
94. akty

In [8]:
y = df['labels']

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.77      0.75      4568
           1       0.84      0.80      0.82      6567

    accuracy                           0.79     11135
   macro avg       0.78      0.79      0.78     11135
weighted avg       0.79      0.79      0.79     11135



In [12]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print("Logistic Regression")
print(classification_report(y_test, y_pred_lr))

Logistic Regression
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      4568
           1       0.82      0.83      0.82      6567

    accuracy                           0.79     11135
   macro avg       0.79      0.79      0.79     11135
weighted avg       0.79      0.79      0.79     11135



In [13]:
import pickle
from google.colab import drive

# Google Drive'ı bağla
drive.mount('/content/drive')

# Kayıt klasörü
save_path = "/content/drive/MyDrive/Colab Notebooks/Research Techniques II/LogReg"

# TF-IDF vectorizer'ı kaydet
with open(f"{save_path}/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

# LogisticRegression modelini kaydet (lr_model olarak eğitilmişse)
with open(f"{save_path}/logreg_model.pkl", "wb") as f:
    pickle.dump(lr_model, f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
